In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
f = open('accessfiles.py', 'wb')
f.write(streaming_body_1.read())


import accessfiles

In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8 MB 12 kB/s s eta 0:00:01|████▌                           | 30.5 MB 14.2 MB/s eta 0:00:14               | 68.2 MB 62.2 MB/s eta 0:00:03�█▎ | 206.1 MB 60.6 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 55.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=f1d6b4cd439e24b63f4d1d2972bfc1a0c4070cc6c9b78df1349fc2cf10eb591a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [4]:
from zipfile import ZipFile
import time
import numpy as np
from operator import add

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

from accessfiles import load_files

load_files()

In [14]:
movies_rdd=sc.textFile("movies.dat")
genre=movies_rdd.map(lambda row:row.split("::")[2])
flat_genre=genre.flatMap(lambda row: row.split("|"))
genre_ctvec=flat_genre.map(lambda genre:(genre,1))
genre_count=genre_ctvec.reduceByKey(lambda x,y:x+y)
genre_sort= genre_count.sortByKey()
genre_sort_df = genre_sort.toDF().toPandas()
genre_sort_df.columns = ['Genre','No. of movies']


In [15]:
from accessfiles import create_download_link
create_download_link(genre_sort_df, filename='Movies_by_genre.csv')